<a href="https://colab.research.google.com/github/Parth-G27/Pytorch-work/blob/main/pytorch_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PyTorch WorkFlow 
#### 1. Get Data ready
#### 2. Build or Pick pretrained model -> Pick loss function & optimizer -> Build a training loop -> 
#### 3. Fit the model to the data (Training) and make predictions
#### 4. Evaluate the model
#### 5. Improve through experimentation 
#### 6. Save and relode the trained model

In [1]:
import torch
from torch import nn
#import matplotlib.pyplotas as plt